<a href="https://colab.research.google.com/github/mheidari98/Movie-Recommender-Systems/blob/main/Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB :
    from google.colab import drive
    drive.mount('/content/drive')

if IN_COLAB :
  Dataset_path = "/content/drive/MyDrive/Colab Notebooks/DataSets/"
else :
  Dataset_path = "./DataSets/"

ml_path = Dataset_path + "ml-latest/"

## Installing required libraries

In [2]:
if IN_COLAB :
    !pip3 install jenkspy

In [3]:
## Imports

In [4]:
# %matplotlib notebook

# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from xgboost import XGBRegressor

import os
import pickle
import wordcloud as wc
import scipy.sparse as sparse
import xgboost as xgb
import random
import statsmodels.tools.tools as stattools
from itertools import combinations

import sklearn
from sklearn import tree
from sklearn.model_selection import KFold,train_test_split
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

## load datasets

In [5]:
ratings = pd.read_csv( Dataset_path + "ratings_2004_75p.csv")

In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4668900 entries, 0 to 4668899
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 142.5 MB


### just keep rating that user and movie are upper than threshhold

In [7]:
min_movie_ratings = 20
filter_Movies = ratings['movieId'].value_counts() > min_movie_ratings
filter_Movies = filter_Movies[filter_Movies].index.tolist()

min_user_ratings = 20
filter_users = ratings['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

print('The original data frame shape:\t{}'.format(ratings.shape))
ratings = ratings[(ratings['movieId'].isin(filter_Movies)) & (ratings['userId'].isin(filter_users))]
print('The new data frame shape:\t{}'.format(ratings.shape))

The original data frame shape:	(4668900, 4)
The new data frame shape:	(4019581, 4)


In [8]:
colName = ratings.movieId.unique()
colCount = len( colName )
colCount

5353

In [9]:
rowName = ratings.userId.unique()
rowCount = len( rowName )
rowCount

49162

In [10]:
# del df

In [11]:
%%time
df = pd.crosstab(ratings.userId, ratings.movieId)

Wall time: 3min 52s


In [12]:
# df = pd.DataFrame(np.zeros((rowCount, colCount)),index=rowName , columns=colName)
# df.info()


# %%time #==> 5min
# for uid in df_new.userId.unique():
#     iids = df_new[df_new.userId == uid].movieId.unique()
#     for iid in iids:
#         df.loc[ uid ,  iid ] = 1


# %%time #==> 12min
# for i in range(len(df_new)):
#     df.loc[ df_new.iloc[i].userId ,  df_new.iloc[i].movieId ] = 1

In [13]:
df

movieId,1,2,3,4,5,6,7,8,9,10,...,8973,8981,8983,8984,8998,8999,9018,27706,30812,30825
userId,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
10,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
31,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283208,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283215,0,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
283218,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49162 entries, 6 to 283224
Columns: 5353 entries, 1 to 30825
dtypes: int64(5353)
memory usage: 2.0 GB


## save to file

In [15]:
df.to_csv(Dataset_path +'AssociationRules.csv', index=False)
del df

## read from file

In [16]:
movies = pd.read_csv( ml_path + "movies.csv")

In [17]:
df = pd.read_csv( Dataset_path + "AssociationRules.csv")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49162 entries, 0 to 49161
Columns: 5353 entries, 1 to 30825
dtypes: int64(5353)
memory usage: 2.0 GB


In [19]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,8973,8981,8983,8984,8998,8999,9018,27706,30812,30825
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
# columnsdf = pd.DataFrame( df.columns, columns=['movieId'] )
# columnsdf.movieId = columnsdf.movieId.astype('int64')
# df.columns = pd.merge( columnsdf, movies , how='inner' ).title.values

In [21]:
if IN_COLAB :
  !pip install mlxtend --upgrade

In [22]:
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

In [23]:
# frq_items_apriori = apriori(df, min_support = 0.1, use_colnames = True)
# frq_items_apriori.sort_values(by=['support'], inplace=True)
# frq_items_apriori

In [24]:
%%time
frq_items_fpgrowth = fpgrowth(df, min_support = 0.1, use_colnames = True)

Wall time: 40 s


In [25]:
frq_items_fpgrowth.sort_values(by=['support'], inplace=True, ascending=False)

In [26]:
frq_items_fpgrowth

,support,itemsets
0,0.444001,(593)
1,0.424413,(318)
2,0.420345,(296)
15,0.383813,(527)
41,0.381473,(356)
...,...,...
874,0.100159,"(1210, 2028)"
969,0.100098,"(1617, 527)"
875,0.100098,"(589, 2028)"
388,0.100077,"(1196, 1198, 527)"


In [27]:
%%time
rules = association_rules(frq_items_fpgrowth, metric ="lift", min_threshold = 1)

Wall time: 279 ms


In [28]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(593),(296),0.444001,0.420345,0.270839,0.609996,1.451180,0.084205,1.486281
1,(296),(593),0.420345,0.444001,0.270839,0.644326,1.451180,0.084205,1.563226
2,(593),(318),0.444001,0.424413,0.268988,0.605827,1.427447,0.080548,1.460241
3,(318),(593),0.424413,0.444001,0.268988,0.633789,1.427447,0.080548,1.518245
4,(296),(318),0.420345,0.424413,0.255828,0.608614,1.434012,0.077428,1.470636
...,...,...,...,...,...,...,...,...,...
3053,(1196),"(1198, 527)",0.307616,0.140902,0.100077,0.325332,2.308934,0.056734,1.273365
3054,(1198),"(1196, 527)",0.303527,0.135593,0.100077,0.329715,2.431657,0.058921,1.289611
3055,(527),"(1196, 1198)",0.383813,0.204853,0.100077,0.260745,1.272838,0.021452,1.075606
3056,(1208),(858),0.146516,0.277186,0.100077,0.683049,2.464228,0.059465,2.280522


In [29]:
# https://stackoverflow.com/questions/1482308
from itertools import chain, combinations
def powerset(iterable):
    allSubList = []
    for r in range(1,len(iterable)+1):
        allSubList += [*combinations(iterable, r)]
    return allSubList

In [30]:
def recommender(rules, ratings, movie_metadata, uid):
    finalIndex = set()
    
    seen = set( ratings[ratings.userId == uid].movieId.unique() )
    
    for index, row in rules.iterrows():
        if set(map(int, row['antecedents'])).issubset(seen) and (not set(map(int, row['consequents'])).issubset(seen)) :
            finalIndex.add(index)
            

    df = rules.loc[finalIndex].sort_values(by=['support'], ascending=False).iloc[:5]

    lst = list(chain( *df['consequents'].values.tolist() ) )
    items = set().union(*lst)
    
    return  movie_metadata[ movie_metadata.movieId.isin( list(map(int, items)) ) ]

In [31]:
recommender(rules, ratings, movies, 6)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
